In [1]:
import pandas as pd
import numpy as np
from scipy import stats

from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.metrics import r2_score , mean_squared_error
from sklearn.metrics import accuracy_score ,mean_absolute_error
import warnings
warnings.filterwarnings("ignore")

# Load the data

### Testing on 10000 data is time Consuming so better to test on 1000 data set and if prediction is best then change data size

In [35]:
x = pd.read_csv("../trgc10000.csv",nrows = 10000)
x = x.iloc[:,1:]
y = pd.read_csv("../srlbininfo_10000",sep ="\t",header =None,nrows = 10000)
y = pd.DataFrame(y.iloc[:,1])
y.columns = ["GC"]

df = pd.concat([y,x],axis =1)

y = pd.DataFrame(df.loc[:,"GC"])
x = df.iloc[:,1:]  #    

#Caution X Column up to 8000 above that outlier and y.GC Less than 

## Always Check the data shape , df , x ,y

In [36]:
df.shape,x.shape , y.shape

((10000, 10333), (10000, 10332), (10000, 1))

In [37]:
display(x.head(2))
display(y.head(2))

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V10323,V10324,V10325,V10326,V10327,V10328,V10329,V10330,V10331,V10332
0,0.879025,0.687329,0.983194,0.970820,0.900728,0.735702,0.960757,1.074772,1.597032,1.156174,...,0,0,0,0,0,0,0,0,0,0
1,0.869801,0.620215,1.008849,0.926203,0.884105,0.884811,1.020117,0.987255,1.335856,1.170143,...,0,0,0,0,0,0,0,0,0,0


,GC
0,19.052706
1,17.772070


In [50]:
y.tail()

,GC
9995,7.962095
9996,3.267010
9997,11.143143
9998,14.716573
9999,14.347694


In [51]:
x_train= None;x_test = None;y_train = None;y_test = None
### 5 Regression 
clf_ridge = None ; y_pred_ridge = None;
clf_lasso = None ; y_pred_lasso = None
regr_elasticNet= None ; y_pred_elasticNet = None
regr_enetcv = None ;y_pred_elasticnetcv = None 
clf_bay = None ; y_pred_bay = None ;

In [6]:
def train_test(size = 0.20 , random_state=50,shuffle = True):
    "size = train size and random state is how the data are mixed and shuffle in to do mixing or not  "
    from sklearn.model_selection import train_test_split
    global x_train,x_test,y_train, y_test
    x_train= None; x_test = None; y_train = None; y_test = None
    x_train,x_test,y_train, y_test = train_test_split(x,y,test_size= size,random_state = random_state , shuffle = shuffle)
    
    #return x_train,x_test,y_train, y_test 

In [7]:
#train_test() # Calling The Function with default Parameter
#print(y_train.shape)
#display(y_train.head(2))
#print(x_train.shape)
#display(x_train.head())

##  Create a Function for Lasso

In [8]:
#Create a Lasso Model to automatically apply in the loop

def lasso( alpha=0.5, fit_intercept=True, normalize=False, precompute=False, copy_X=True, 
                      max_iter=1000, tol=0.0001, warm_start=False, positive=False,selection='cyclic'):
    
    "Just Pass Related values only"
    
    from sklearn.linear_model import Lasso
    from sklearn.model_selection import train_test_split
    global x_train,x_test,y_train, y_test , clf_lasso , y_pred_lasso
    
    clf_lasso = Lasso(alpha=alpha, fit_intercept=fit_intercept , normalize=normalize, precompute=precompute, copy_X=copy_X, 
                      max_iter=max_iter, tol=tol, warm_start=warm_start, positive=positive,selection=selection).fit(x_train,y_train)
    
    y_pred_lasso = pd.DataFrame(clf_lasso.predict(x_test))
    name = "Lasso"
    # *****************************************************************
    
#    print(name +" Coefficient ", clf_lasso.coef_) 
#    print(name + " Intercept " , clf_lasso.intercept_,"\n") ##
#    print("****"*5+"Accuracy Test Model Fitting "+"****"*5)
#    print(name + "Score for train data set :",clf_lasso.score(x_train,y_train))
#    print(name + "Score for test data Set",clf_lasso.score(x_test,y_test))
#
#    print(name + "Score for Predictecd data Set",clf_lasso.score(x_test,y_pred_lasso)) ##
#    
#    #print("\n"+"****"*5+"R2 Score"+"****"*5)
#    #print(name + "R2 Score for test is = ", r2_score(y_test, y_pred_lasso)) ##
#    
#    print("\n"+"****"*5+"Mean Absolute Error"+"****"*5)
#    print(name+ "Mean Absolute Error of Test = ", mean_absolute_error(y_test,y_pred_lasso)) ##
#    print(name +"Mean Absolute Error on Train",mean_absolute_error(y_train,pd.DataFrame(clf_lasso.predict(x_train))))
#    
#    print("\n"+"****"*5+" Mean Squared Error"+"****"*5)
#    print(name + "Mean Squared Error of Test = ",mean_squared_error(y_test,y_pred_lasso)) ##
#    print(name+"Mean Squared Error on Train = ",mean_squared_error(y_train,pd.DataFrame(clf_lasso.predict(x_train))))
#    
#    print("\n"+"****"*5+" Co Relation"+"****"*5)
#    print(name + "Correlation Test Data ",stats.pearsonr(y_test["GC"],y_pred_lasso[0])[0])
#    print(name + "Correlation Original Data ",stats.pearsonr(y_train["GC"],pd.DataFrame(clf_lasso.predict(x_train))[0])[0])
    
    clf_lasso.score(x_test,y_test) ,stats.pearsonr(y_test.GC,pd.DataFrame(clf_lasso.predict(x_test))[0])[0]

## Function For Ridge Regression

In [9]:
#Create a Model to automatically apply in the loop

def ridge( alpha = 0.01,copy_X=True,fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001):
    "Just Pass alpha value only"
    
    from sklearn.linear_model import Ridge
    from sklearn.model_selection import train_test_split
    global x_train,x_test,y_train, y_test , clf_ridge , y_pred_ridge
    
    clf_ridge = Ridge(alpha=alpha,copy_X=copy_X,fit_intercept=fit_intercept, max_iter=max_iter, normalize=normalize,
      random_state=random_state, solver=solver, tol=tol).fit(x_train,y_train)
    
    y_pred_ridge = pd.DataFrame(clf_ridge.predict(x_test))
    name = "Ridge"
    # *****************************************************************
    print ("Ridge Regression")
   # print(name +" Coefficient ", clf_ridge.coef_) 
#    print(name + " Intercept " , clf_ridge.intercept_,"\n") ##
#    print("****"*5+"Accuracy Test Model Fitting "+"****"*5)
#    print(name + "Score for Train data set :",clf_ridge.score(x_train,y_train))
#    print(name + "Score for Test data Set",clf_ridge.score(x_test,y_test))
    
    #print(name + "Score for Train data set :",clf_ridge.score(x_train,y_train),"Test data Set =",clf_ridge.score(x_test,y_test))
    #Return the coefficient of determination R^2 of the prediction.
    
    

#    print(name + "Score for Predictecd data Set",clf_ridge.score(x_test,y_pred_ridge)) ##
    
    #print("\n"+"****"*5+"R2 Score"+"****"*5)
    #print(name + "R2 Score for test is = ", r2_score(y_test, y_pred_ridge)) ##
    
#    print("\n"+"****"*5+"Mean Absolute Error"+"****"*5)
#    print(name+ "Mean Absolute Error of Test = ", mean_absolute_error(y_test,y_pred_ridge)) ##
#    print(name +"Mean Absolute Error on Train",mean_absolute_error(y_train,pd.DataFrame(clf_ridge.predict(x_train))))
    
#    print("\n"+"****"*5+" Mean Squared Error"+"****"*5)
#    print(name + "Mean Squared Error of Test = ",mean_squared_error(y_test,y_pred_ridge)) ##
#    print(name+"Mean Squared Error on Train = ",mean_squared_error(y_train,pd.DataFrame(clf_ridge.predict(x_train))))
    
#    print("\n"+"****"*5+" Co Relation"+"****"*5)
#    print(name + "Correlation Test Data ",stats.pearsonr(y_test["GC"],y_pred_ridge[0])[0])
#    print(name + "Correlation Original Data ",stats.pearsonr(y_train["GC"],pd.DataFrame(clf_ridge.predict(x_train))[0])[0])
    
    #print("Correlation = ",stats.pearsonr(y_train["GC"],pd.DataFrame(clf_ridge.predict(x_train))[0])[0])
    
    return clf_ridge.score(x_test,y_test) ,stats.pearsonr(y_test.GC,pd.DataFrame(clf_ridge.predict(x_test))[0])[0]
    

## Create A Function for ElasticNet

In [10]:
def enet(alpha=1.0, l1_ratio=0.5, fit_intercept=True, normalize=False,precompute=False, max_iter=1000,
         copy_X=True, tol=0.0001, warm_start=False, positive=False, random_state=None, selection='cyclic'):
    
    from sklearn.linear_model import ElasticNet
    global x_train,x_test,y_train, y_test , regr_elasticNet, y_pred_elasticNet

    name = "Elastic net--> "

    regr_elasticNet = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, fit_intercept=fit_intercept, normalize=normalize,
                                 precompute=precompute, max_iter=max_iter,copy_X=copy_X, tol=tol, warm_start=warm_start, 
                                 positive=positive, random_state=random_state, selection=selection)
    
    regr_elasticNet.fit(x_train , y_train)

    y_pred_elasticNet =pd.DataFrame(regr_elasticNet.predict(x_test))
    print("Elastic Net")
    
#    print(name +" Coefficient ", regr_elasticNet.coef_) 
#    print(name + " Intercept " , regr_elasticNet.intercept_,"\n") ##
#    print("****"*5+"Accuracy Test Model Fitting "+"****"*5)
#    print(name + "Score for train data set :",regr_elasticNet.score(x_train,y_train))
#    #Return the coefficient of determination R^2 of the prediction.
#    print(name + "Score for test data Set",regr_elasticNet.score(x_test,y_test))
#    print(name + "Score for Predictecd data Set",regr_elasticNet.score(x_test,y_pred_elasticNet)) ##
#    #print("\n"+"****"*5+"R2 Score"+"****"*5)
#    #print(name + "R2 Score for test is = ", r2_score(y_test,y_pred_sgd)) ##
#    print("\n"+"****"*5+"Mean Absolute Error"+"****"*5)
#    print(name+ "Mean Absolute Error of Test = ", mean_absolute_error(y_test,y_pred_elasticNet)) ##
#    print(name +"Mean Absolute Error on Train",mean_absolute_error(y_train,pd.DataFrame(regr_elasticNet.predict(x_train))))
#    print("\n"+"****"*5+" Mean Squared Error"+"****"*5)
#    print(name + "Mean Squared Error of Test = ",mean_squared_error(y_test,y_pred_elasticNet)) ##
#    print(name+"Mean Squared Error on Train = ",mean_squared_error(y_train,pd.DataFrame(regr_elasticNet.predict(x_train))))
#    print("\n"+"****"*5+" Co Relation"+"****"*5)
#    print(name + "Correlation Test Data ",stats.pearsonr(y_test["GC"],y_pred_elasticNet[0])[0])
#    print(name + "Correlation Original Data ",stats.pearsonr(y_train["GC"],pd.DataFrame(regr_elasticNet.predict(x_train))[0])[0])
    
    return regr_elasticNet.score(x_test,y_test) ,stats.pearsonr(y_test.GC,pd.DataFrame(regr_elasticNet.predict(x_test))[0])[0]

## Function for Elastic NetCV  **** glmnet in R ***

In [47]:
def enet_cv(l1_ratio=0.5, eps=0.001, n_alphas=100, alphas=None, fit_intercept=True, normalize=False, precompute='auto',
            max_iter=1000, tol=0.0001, cv=None, copy_X=True, verbose=0, n_jobs=None, positive=False, random_state=None,
            selection='cyclic'):
    
    from sklearn.linear_model import ElasticNetCV
    global x_train,x_test,y_train, y_test , regr_enetcv , y_pred_elasticnetcv
    name = "ElasticnetCV --> "

    regr_enetcv = ElasticNetCV(l1_ratio=l1_ratio, eps=eps, n_alphas=n_alphas, alphas=alphas, fit_intercept=fit_intercept,
                               normalize=normalize,precompute=precompute, max_iter=max_iter, tol=tol, cv=cv,copy_X=copy_X,
                               verbose=verbose, n_jobs=n_jobs,positive=positive, random_state=random_state, selection=selection)
    
    regr_enetcv.fit(x_train,y_train)

    y_pred_elasticnetcv =pd.DataFrame(regr_enetcv.predict(x_test))
    print("Elastic Net CV")
#    print(name +" Coefficient ", regr_enetcv.coef_) 
#    print(name + " Intercept " , regr_enetcv.intercept_,"\n") ##
#    
#    print("****"*5+"Accuracy Test Model Fitting "+"****"*5)
#    print(name + "Score for train data set :",regr_enetcv.score(x_train,y_train))
#    #Return the coefficient of determination R^2 of the prediction.
#    print(name + "Score for test data Set",regr_enetcv.score(x_test,y_test))
#    print(name + "Score for Predictecd data Set",regr_enetcv.score(x_test,y_pred_elasticnetcv)) ##
#    
#    #print("\n"+"****"*5+"R2 Score"+"****"*5)
#    #print(name + "R2 Score for test is = ", r2_score(y_test,y_pred_elasticnetcv)) ##
#    
#    print("\n"+"****"*5+"Mean Absolute Error"+"****"*5)
#    print(name+ "Mean Absolute Error of Test = ", mean_absolute_error(y_test,y_pred_elasticnetcv)) ##
#    print(name +"Mean Absolute Error on Train",mean_absolute_error(y_train,pd.DataFrame(regr_enetcv.predict(x_train))))
#    print("\n"+"****"*5+" Mean Squared Error"+"****"*5)
#    print(name + "Mean Squared Error of Test = ",mean_squared_error(y_test,y_pred_elasticnetcv)) ##
#    print(name+"Mean Squared Error on Train = ",mean_squared_error(y_train,pd.DataFrame(regr_enetcv.predict(x_train))))
#    print("\n"+"****"*5+" Co Relation"+"****"*5)
#    print(name + "Correlation Test Data ",stats.pearsonr(y_test["GC"],y_pred_elasticnetcv[0])[0])
#    print(name + "Correlation Original Data ",stats.pearsonr(y_train["GC"],pd.DataFrame(regr_enetcv.predict(x_train))[0])[0])
    
    return regr_enetcv.score(x_test,y_test) ,stats.pearsonr(y_test.GC,pd.DataFrame(regr_enetcv.predict(x_test))[0])[0]

## Function For Bayesian 

In [12]:
def bayesian_ridge(n_iter=300, tol=0.001, alpha_1=1e-06, alpha_2=1e-06, lambda_1=1e-06, lambda_2=1e-06,
                compute_score=False, fit_intercept=True, normalize=False,copy_X=True, verbose=False):
 
    from sklearn.linear_model import BayesianRidge
    global x_train,x_test,y_train, y_test ,clf_bay , y_pred_bay
    
    name = "BayesianRidge"
    clf_bay = BayesianRidge(n_iter=n_iter, tol=tol, alpha_1=alpha_1, alpha_2=alpha_2, lambda_1=lambda_1, lambda_2=lambda_2,
                        compute_score=compute_score,fit_intercept=fit_intercept,normalize=normalize,copy_X=copy_X,
                            verbose=verbose).fit(x_train , y_train)
    
    y_pred_bay = pd.DataFrame(clf_bay.predict(x_test))
    print("Bayesian Ridge")
#    print(name +"Coefficient ",clf_bay.coef_) 
#    print(name + " Intercept " , clf_bay.intercept_,"\n") ##
    print("****"*5+"Accuracy Test Model Fitting "+"****"*5)
    print(name + "Score for train data set :",clf_bay.score(x_train,y_train))
    #Return the coefficient of determination R^2 of the prediction.
    print(name + "Score for test data Set",clf_bay.score(x_test,y_test))
#    print(name + "Score for Predictecd data Set",clf_bay.score(x_test,y_pred_bay)) ##
    #print("\n"+"****"*5+"R2 Score"+"****"*5)
    #print(name + "R2 Score for test is = ", r2_score(y_test,y_pred_bay)) ##
#    print("\n"+"****"*5+"Mean Absolute Error"+"****"*5)
#    print(name+ "Mean Absolute Error of Test = ", mean_absolute_error(y_test,y_pred_bay)) ##
#    print(name +"Mean Absolute Error on Train",mean_absolute_error(y_train,pd.DataFrame(clf_bay.predict(x_train))))
#    print("\n"+"****"*5+" Mean Squared Error"+"****"*5)
#    print(name + "Mean Squared Error of Test = ",mean_squared_error(y_test,y_pred_bay)) ##
#    print(name+"Mean Squared Error on Train = ",mean_squared_error(y_train,pd.DataFrame(clf_bay.predict(x_train))))
#    print("\n"+"****"*5+" Co Relation"+"****"*5)
#    print(name + "Correlation Test Data ",stats.pearsonr(y_test["GC"],y_pred_bay[0])[0])
#    print(name + "Correlation Original Data ",stats.pearsonr(y_train["GC"],pd.DataFrame(clf_bay.predict(x_train))[0])[0])
   
    return clf_bay.score(x_test,y_test) ,stats.pearsonr(y_test.GC,pd.DataFrame(clf_bay.predict(x_test))[0])[0]

In [52]:
# Default 
train_test(size = 0.2 , random_state=5,shuffle = True)

### Call the Ridge Function in  with different Parameter

In [14]:
train_test(size = 0.2 , random_state=5,shuffle = True)

In [70]:
#We have 2 Function 
train_test(size = 0.2 , random_state=5,shuffle = True)
#***********************************************************
print(ridge(alpha=0.9,copy_X=True,fit_intercept=True, max_iter=2,
            normalize=True,tol= 0.01,solver="svd"))

Ridge Regression
(0.755343190622171, 0.8730904560557753)


In [16]:
train_test(size = 0.2 , random_state=580,shuffle = True)
#***********************************************************
print(ridge(alpha=0.9,copy_X=True,fit_intercept=True, max_iter=1, normalize=True))

Ridge Regression
(0.5932183083966058, 0.8277061342736141)


In [17]:
# 'sparse_cg', 'cholesky', 'svd' 'lsqr', 'sag' or 'saga'
train_test(size = 0.2 , random_state=580,shuffle = True)
#***********************************************************
print(ridge(alpha=0.9,copy_X=True,fit_intercept=True, max_iter=1, normalize=True))

Ridge Regression
(0.5932183083966058, 0.8277061342736141)


#### Bayesian For Testing

In [18]:
#train_test(size = 0.2 , random_state=5,shuffle = True)
#***********************************************************
bayesian_ridge(n_iter=300, tol=0.0001, alpha_1=1e-06, alpha_2=1e-06, lambda_1=1e-06, lambda_2=1e-06,compute_score=True,
               fit_intercept=True, normalize=True,copy_X=True, verbose=False)

Bayesian Ridge
********************Accuracy Test Model Fitting ********************
BayesianRidgeScore for train data set : 0.9999999999999782
BayesianRidgeScore for test data Set 0.6121500177670123


(0.6121500177670123, 0.8280507616474337)

## Elastic Net Regression  (Time Consuming)

In [19]:
#train_test(size = 0.2 , random_state=580,shuffle = True)
#***********************************************************
enet(alpha=1.0, l1_ratio=0.1, fit_intercept=True, normalize=False,precompute=False, max_iter=1000,
         copy_X=True, tol=0.0001, warm_start=False, positive=False, random_state=None, selection='cyclic')

Elastic Net


(0.009198462561607279, 0.1641535794219543)

In [ ]:
#train_test(size = 0.2 , random_state=580,shuffle = True)
#***********************************************************
enet(alpha=1.0, l1_ratio=0.1, fit_intercept=True, normalize=False,precompute=False, max_iter=1000,
         copy_X=True, tol=0.0001, warm_start=False, positive=False, random_state=None, selection='cyclic')

## Bayesian Ridge Regression

In [20]:
train_test(size = 0.05 , random_state=580,shuffle = True)
#***********************************************************
bayesian_ridge(n_iter=100, tol=0.0001, alpha_1=1e-03, alpha_2=1e-03, lambda_1=1e-03, lambda_2=1e-03,compute_score=True,
               fit_intercept=True, normalize=True,copy_X=True, verbose=False)

Bayesian Ridge
********************Accuracy Test Model Fitting ********************
BayesianRidgeScore for train data set : 0.9999999800927856
BayesianRidgeScore for test data Set 0.5850289163426128


(0.5850289163426128, 0.780127619819467)

In [21]:
train_test(size = 0.20 , random_state=580,shuffle = True)
#***********************************************************
bayesian_ridge(n_iter=100, tol=0.0001, alpha_1=1e-06, alpha_2=1e-06, lambda_1=1e-06, lambda_2=1e-06,compute_score=True,
               fit_intercept=True, normalize=True,copy_X=True, verbose=False)

Bayesian Ridge
********************Accuracy Test Model Fitting ********************
BayesianRidgeScore for train data set : 0.9999999999999782
BayesianRidgeScore for test data Set 0.6121500177670123


(0.6121500177670123, 0.8280507616474337)

In [22]:
#Verbose True May take long time

train_test(size = 0.10 , random_state=580,shuffle = True)
#***********************************************************
bayesian_ridge(n_iter=100, tol=0.0001, alpha_1=1e-06, alpha_2=1e-06, lambda_1=1e-06, lambda_2=1e-06,compute_score=True,
               fit_intercept=True, normalize=True,copy_X=True, verbose=True) #verbose True

Convergence after  24  iterations
Bayesian Ridge
********************Accuracy Test Model Fitting ********************
BayesianRidgeScore for train data set : 0.9999999999999784
BayesianRidgeScore for test data Set 0.6058995946757333


(0.6058995946757333, 0.8086440723732387)

In [23]:
train_test(size = 0.10 , random_state=580,shuffle = True)
#***********************************************************
bayesian_ridge(n_iter=100, tol=0.0001, alpha_1=1e-06, alpha_2=1e-06, lambda_1=1e-06, lambda_2=1e-06,compute_score=True,
               fit_intercept=True, normalize=True,copy_X=True, verbose=False) #verbose True

Bayesian Ridge
********************Accuracy Test Model Fitting ********************
BayesianRidgeScore for train data set : 0.9999999999999784
BayesianRidgeScore for test data Set 0.6058995946757333


(0.6058995946757333, 0.8086440723732387)

## Elastic Net CV ** Glmnet ** R
#### take long Time

In [24]:
train_test(size = 0.2 , random_state=580,shuffle = True)
##***********************************************************
enet_cv(l1_ratio=0.5, eps=0.001, n_alphas=10, alphas=None, fit_intercept=True, normalize=False, precompute='auto',
            max_iter=1000, tol=0.0001, cv=5, copy_X=True, verbose=0, n_jobs=None, positive=False, random_state=None,
            selection='cyclic')

Elastic Net CV


(0.6325953586863764, 0.815460317827753)

In [64]:
train_test(size = 0.001 , random_state=580,shuffle = True)
##***********************************************************
enet_cv(l1_ratio=0.5, eps=0.001, n_alphas=10, alphas=None, fit_intercept=True, normalize=False, precompute='auto',
            max_iter=10, tol=0.0001, cv=15, copy_X=True, verbose=0, n_jobs=None, positive=False, random_state=None,
            selection='random')

Elastic Net CV


(0.7987677710871236, 0.9664110973023888)

In [68]:
ffy_enetcv = pd.DataFrame(regr_enetcv.predict(test_x))

In [69]:
ffy_enetcv.to_csv("../enet_pred.csv",index=None)

## Testing 900 Data Sets with Our Developed Model

In [26]:
y_pred_ridge.head()
test_x = pd.read_csv("../trgc900.csv")
test_x = test_x.iloc[:,1:]
#
test_y = pd.read_csv("../ffyrgc900.csv")
test_y = test_y.iloc[:,1:]*100

In [27]:
print("Correlation with Ridge Reg Pred",stats.pearsonr(test_y["V1"],pd.DataFrame(clf_ridge.predict(test_x))[0])[0])
print("Correlation with Bay Reg Pred",stats.pearsonr(test_y["V1"],pd.DataFrame(clf_bay.predict(test_x))[0])[0])

Correlation with Ridge Reg Pred 0.7936803552751299
Correlation with Bay Reg Pred 0.7998115841953239


In [71]:
pd.DataFrame(clf_ridge.predict(test_x))[0].to_csv("ridge_pred.csv",index=False)

In [63]:
#pd.DataFrame(clf_ridge.predict(test_x))[0]
df.shape

(10000, 10333)